In [1]:
import argparse
import torch
import random
import os
import wandb
import warnings
import numpy as np

import dex_envs
from rl_modules.utils import *
from rl_modules.ddpg_agent import ddpg_agent
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
MESH_NAMES=['knife']
ALL_TRAIN=['knife']
ALL_TEST=['fork']

In [3]:
parser = argparse.ArgumentParser()
args =parser.parse_args(args=[])

In [4]:
args.train_names = []#the environement name
args.test_names = []#test_names
args.n_cycles = 40000#the times to collect samples per epoch
args.n_batches = 4#the times to update the network
args.seed = 125
args.replay_strategy = "future"# the HER strategy
args.save_dir = "dex_logs/"#the path to save the models
args.noise_eps = 0.2#noise eps
args.random_eps = 0.3 # random eps
args.buffer_size = int(1e6) #the size of the buffer

args.replay_k = 4 #ration to be replace
args.clip_obs = 200 #the clip ratio
args.batch_size= 256# the sample batch size
args.gamma =0.98 #the discount factor
args.action_l2 = 1#l2 reg
args.lr_actor = 0.001281 #the learning rate of the actor
args.lr_critic = 0.000884
args.polyak = 0.95 #the average coefficient
args.n_test_rollouts = 2 # the number of tests
args.clip_range = 5 # the clip ra

In [5]:
"""
parser.add_argument('--no_cuda', action='store_true',
                        help='if use gpu do the acceleration')
parser.add_argument('--serial', action="store_true",
                        help="do not use multiprocessing")
parser.add_argument('--no_save', action="store_true",
                        help="do not save anything")
parser.add_argument('--eval', action="store_true",
                        help="only eval the network once")
parser.add_argument('--finetune_pointnet', action="store_true",
                    help="allow finetuning in dagger training")
parser.add_argument('--fresh', action="store_true",
                    help="ignore saved checkpoint; start from scratch")
parser.add_argument('--no_save_buffer', action="store_true",
                    help="do not save buffer when saving general checkpoint (to avoid memory error if buffer is too big)")
parser.add_argument('--point_cloud', action="store_true",
                    help="enable using pointnet + MLP")


"""
    

'\nparser.add_argument(\'--no_cuda\', action=\'store_true\',\n                        help=\'if use gpu do the acceleration\')\nparser.add_argument(\'--serial\', action="store_true",\n                        help="do not use multiprocessing")\nparser.add_argument(\'--no_save\', action="store_true",\n                        help="do not save anything")\nparser.add_argument(\'--eval\', action="store_true",\n                        help="only eval the network once")\nparser.add_argument(\'--finetune_pointnet\', action="store_true",\n                    help="allow finetuning in dagger training")\nparser.add_argument(\'--fresh\', action="store_true",\n                    help="ignore saved checkpoint; start from scratch")\nparser.add_argument(\'--no_save_buffer\', action="store_true",\n                    help="do not save buffer when saving general checkpoint (to avoid memory error if buffer is too big)")\nparser.add_argument(\'--point_cloud\', action="store_true",\n                    he

In [6]:
args.no_cuda = True
args.serial = True
args.no_save = False
args.finetune_pointnet = False
args.fresh = True
args.no_save_buffer =False
args.point_cloud = True

In [7]:
args.num_rollouts = 1 #the rollout per cycle
args.expID = 0
args.eval_freq = 50 #number of cycles between eval and logging
args.video_count = 1 # number of videos to record during the logging
args.load_path = None# expid to load path
args.pointnet_load_path = 0
args.epoch_per_cycle = 22.966#alternative way to specify n_batches ;
#it means the number of epochs to train over the size of the neewly collected data; 
#if newly collected 1000 transitions,10epochs means training for 100 batches
#with batch size being 100
args.num_parallel_envs=1#number of parallel VecEnv; if not specified, calculate from the available vCPUs')
args.chunk_size=10#if not equal 0, use SubprocChunkVecEnv as proposed in baselines PR 620 with given chunk size')
args.load_cycle=None#the cycle index of the loaded eval checkpoint')
args.layers=4
args.width=768
args.num_points = 128#number of points to sample in mujoco")
args.pointnet_output_dim = 512#output dim of the two stream pointnet feature net")
args.eval=False

In [8]:
#args = parser.parse_args()


In [9]:
args.train_names = args.train_names if args.train_names else ALL_TRAIN
args.test_names = args.test_names if args.test_names else ALL_TEST
assert len(list(set(args.train_names) & set(args.test_names))) == 0, 'cannot have overlapping train/test envs'

# if using eval mode, do not start new training and do not save
if args.eval:
    args.fresh = False
    args.no_save = True

# alternative way to specify how many batches to update per cycle
if args.epoch_per_cycle:
    batches_per_epoch = (args.num_rollouts * 100) / args.batch_size
    batches_total = int(batches_per_epoch * args.epoch_per_cycle)
    print('*** overwriting n_batches from {} to {} ***'.format(args.n_batches, batches_total))
    args.n_batches = batches_total

# cap buffer size at 10M
total_buffer_size = len(args.train_names) * args.buffer_size
if total_buffer_size > int(1e7):
    args.buffer_size = int(1e7) // len(args.train_names)
    print('adjusted total buffer size from {:.4e} to 1e7'.format(
        total_buffer_size))

# require parallization args align with each other
if args.n_cycles > 0 or args.eval:
    if len(args.train_names) > 0:
        assert args.num_parallel_envs % (len(
            args.train_names)) == 0, 'num_parallel_envs must be multiple of num of train envs ({})'.format(len(args.train_names))
        assert args.num_rollouts % args.num_parallel_envs == 0, 'num_rollouts must be multiple of parallel env number ({})'.format(
            args.num_parallel_envs)

    assert args.n_test_rollouts % (len(args.train_names + args.test_names)) == 0, 'n_test_rollouts must be multiple of num of train envs + test_envs ({} + {} = {})'.format(
        len(args.train_names), len(args.test_names), len(args.train_names + args.test_names))
    # make sure n_batches must be multiple of num of train envs
    if len(args.train_names) > 0:
        if args.epoch_per_cycle:
            args.n_batches = (
                args.n_batches // len(args.train_names)) * len(args.train_names)
        else:
            assert args.n_batches % len(
                args.train_names) == 0, 'n_batches must be multiple of num of train envs ({})'.format(len(args.train_names))

*** overwriting n_batches from 4 to 8 ***


In [10]:
def init_callback(args, prefix):
    if not args.eval and not args.fresh:
        resume_mode = 'allow'
    else:
        resume_mode = None
    run_name = '{}_{:04d}'.format(prefix, args.expID)
    wandb.init(name=run_name, id=run_name, resume=resume_mode,
               save_code=True, anonymous="allow")
    wandb.config.update(args, allow_val_change=True)


def log_callback(log_dict):
    wandb.log(log_dict)


def get_env_params(env, args):
    obs = env.reset()
    # close the environment
    params = {'goal': obs['desired_goal'].shape[-1],
              'action': env.action_space.shape[-1],
              'action_max': env.action_space.high[-1],
              'max_timesteps': env._max_episode_steps,
              'obs_to_normalize': obs['minimal_obs'].shape[-1]
              }
    if args.point_cloud:
        params['obs'] = obs['pc_obs'].shape[-1]
    else:
        params['obs'] = params['obs_to_normalize']
    return params


def get_policy_params(env, args):
    obs = env.reset()
    params = dict(state_dim=obs['minimal_obs'].shape[-1] + obs['desired_goal'].shape[-1],
                  action_dim=env.action_space.shape[-1],
                  max_action=env.action_space.high[-1],
                  args=args)
    return params




In [11]:
os.environ["WANDB_SILENT"] = "true"
init_callback=init_callback
log_callback=log_callback

In [12]:
args

Namespace(action_l2=1, batch_size=256, buffer_size=1000000, chunk_size=10, clip_obs=200, clip_range=5, epoch_per_cycle=22.966, eval=False, eval_freq=50, expID=0, finetune_pointnet=False, fresh=True, gamma=0.98, layers=4, load_cycle=None, load_path=None, lr_actor=0.001281, lr_critic=0.000884, n_batches=8, n_cycles=40000, n_test_rollouts=2, no_cuda=True, no_save=False, no_save_buffer=False, noise_eps=0.2, num_parallel_envs=1, num_points=128, num_rollouts=1, point_cloud=True, pointnet_load_path=0, pointnet_output_dim=512, polyak=0.95, random_eps=0.3, replay_k=4, replay_strategy='future', save_dir='dex_logs/', seed=125, serial=True, test_names=['fork'], train_names=['knife'], video_count=1, width=768)

In [13]:
dummy_env = makeEnv((args.train_names + args.test_names)[0], 0, args)()


In [14]:


#args = get_args()
    # create dummy env for accessing spaces attr
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.no_cuda:
    torch.cuda.manual_seed(args.seed)
# get the environment parameters
# assume all envs high-level attributes are the same, use arbitrary one


In [ ]:
env_params = get_env_params(dummy_env, args)
# assume all envs high-level attributes are the same, use arbitrary one
policy_params = get_policy_params(dummy_env, args)
# create the ddpg agent to interact with the environment
trainer = ddpg_agent(args, dummy_env, env_params, policy_params)
init_callback(args=args, prefix=trainer.agent_type)

if args.eval:
    trainer.eval(log_callback=log_callback)
else:
    trainer.learn(log_callback=log_callback)

dummy_env.close()




** creating 1 training vec env.. created (time taken: 0.10 s)! **

** creating 2 eval vec env.. created (time taken: 0.18 s)! **

****************************************
** starting a new run
****************************************
*** successfully loaded pointnet feature model ***


[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


== [EXP0000:C000] success %: 0.000, reward: -100.00, dist: 0.711, env time: 2.80 s across 1 episodes


In [ ]:
args.eval